## Data Import

In [2]:
import pandas as pd
import re
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm, tqdm_notebook, trange
tqdm.pandas()
import pickle

In [3]:
use_col_post = ['category','content_id','user_id','title','tags','post_date','last_update','content']
df_post = pd.read_csv('../src/pantip_car_post.csv',usecols=use_col_post)

# use_col_cmt = ['category','content_id','user_id','comment','tags','post_date','last_update']
# df_cmt = pd.read_csv('../src/pantip_car_comment.csv',usecols=use_col_cmt)

/Users/witchapong/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
# df_post = pd.read_csv('../src/pantip_aviation.csv')
# df_post = df_post.iloc[:,[5,6,9,11,12,15,16,17]]
# df_post.columns = ['category','content_id','user_id','title','tags','post_date','last_update','content']

In [3]:
# major car manufacturer
# Honda: CRV ,BRV, CIVIC, CITY, JAZZ, ACCORD
# Toyota: CHR, ALTIS, YARIS, VIOUS, CAMRY

In [4]:
df_post.head()

,category,content_id,user_id,title,tags,post_date,last_update,content
0,auto,38339573,3300767.0,Scoopi I วิ่งอยู่ดีๆเกิดสำลัก สะดุด แล้วดับกล...,"มอเตอร์ไซค์,Honda Scoopy i,Honda (มอเตอร์ไซค์)",2018-12-06 21:33:54,0000-00-00 00:00:00,Scoopi I วิ่งอยู่ดีๆเกิดสำลัก สะดุด แล้วดับก...
1,auto,38339569,4918714.0,ทำไมรถFormula 1 เครื่องยนต์1.6ลิตร แต่แรงม้า70...,"ไซเบอร์ฟอร์มูล่า,Formula 1,รถยนต์ไฮบริด,Super ...",2018-12-06 21:32:58,0000-00-00 00:00:00,1.6L คือ 1600 CC หรือป่าวถ้าเข้าใจผิดขออภัย แต...
2,auto,38339500,885135.0,จะซื้อรถใหม่ ขอคำแนะนำครับ,"รถยนต์,ดูแลรักษารถยนต์,Toyota Yaris,Mazda 2 Sp...",2018-12-06 21:13:34,0000-00-00 00:00:00,ระหว่าง มาสด้า hi connect 620000กับ ยาริส ตั...
3,auto,38339462,4890089.0,งง กับ Filano hybrid,"Yamaha,มอเตอร์ไซค์",2018-12-06 21:05:53,0000-00-00 00:00:00,คือว่าเรากำลังจะซื้อ Filano hybrid ตัวtop ที่เ...
4,auto,38339443,531171.0,บัตรมอเตอร์ expo เป็นบัตรฟรีหรือต้องเสียตังค์ครับ,รถยนต์,2018-12-06 21:02:19,0000-00-00 00:00:00,NaN


In [5]:
df_post.shape

(417346, 8)

In [6]:
df_post['title-content'] = df_post['title'].fillna('') + ' ' + df_post['content'].fillna('')

In [7]:
# randomly check the text
rand_topic_content = df_post.loc[np.random.randint(0,df_post.shape[0]),'title-content']
print(rand_topic_content)

สอบถามเส้นทางจากอุดมสุข ไปฟิวเจอร์พาร์ค รังสิตค่ะ พอดีวันที่ 29 นี้จะต้องเดินทางไปตรวจร่างกายที่ ศูนย์การแพทย์สมเด็จพระเทพรัตราชสุดาสยามบรมราชกุมารี ที่องค์รักษ์อะค่ะ แล้วไม่เคยเดินทางไปเส้นนั้นเลย  รู้แค่เพียงว่าต้องไปขึ้นรถตู้ที่ฟิวเจอร์ฯ  คุณหมอนัด 9 โมงเช้า ต้องเดินทางโดยรถโดยสารทางไหนที่คะ รถตู้ หรือแทกซี่ ดีคะ รบกวนด้วยนะคะ


## Text cleaning

In [8]:
# prepare token from pantip data
import string
import re

# put everything all together as a function
def clean_msg(msg):
    
    # remove all the html notation---OK
    msg = re.sub( r"(\<(\/)?(\w)*(\d)?\>)",'', msg)
    
    # remove the link---OK
    msg = re.sub(r'https*.*? ',' url ',msg)
    
    # remove the link2---OK
    msg = re.sub(r'https*.*',' url ',msg)

    # remove the link3---OK
    msg = re.sub(r'www*.*? ',' url ',msg)
    
    # remove the link4---OK
    msg = re.sub(r'www*.*',' url ',msg)
    
    # remove the email---OK
    msg = re.sub(r'[\w.-]+@[\w.]+',' email ',msg)
    
    # remove the date---OK
    msg = re.sub(r'\d+\W\d+\W\d+','date ',msg)

    # remove the date2---OK
    msg = re.sub(r'\d+\W\d+\W\d+',' date ',msg)
    
    # remove the time <--- need to check logic
    msg = re.sub(r'\d+\s*:\s*\d+',' time ',msg)
    
    # remove the time2 <--- need to check logic
    msg = re.sub(r'\d+\s*\.\s*\d+',' time ',msg)
    
    # remove the number
    msg = re.sub(r' \d+,*\d*,*\d*,*\d*,*\d*,*\d* ',' number ',msg)
    
    # removing punctuation ---> move to the dataframe function
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),' ',msg)
        
    # remove white blank space
    msg = re.sub(r'\s+',' ',msg)
    
    return msg

In [12]:
df_post['title_content_clean'] = df_post['title-content'].progress_apply(clean_msg)

100%|██████████| 417346/417346 [01:44<00:00, 3993.14it/s]


In [13]:
df_post = df_post.dropna(subset=['post_date','content_id'])
df_post = df_post.reset_index(drop=True)

In [12]:
# df_post['content_id'].progress_apply(lambda x:re.sub(r'\d+','',x) == '')

## Text tokenizing

In [14]:
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie

noncut_list = ['mazda 2','yaris ativ','mazda2']
thai_stop = thai_stopwords()

def tokenize(text):
    
    # change to lower case to normalise English word
    text = text.lower()
    
    # tokenize
    tokens = word_tokenize(text,custom_dict=noncut_list, engine='newmm')
    
    # remove stopwords
    tokens = [tkn for tkn in tokens if not tkn in thai_stop]

    
    return tokens

In [24]:
tokens_post = []
for i in trange(df_post.shape[0]):
    tokens_post.append(tokenize(df_post.iloc[i]['title_content_clean']))

100%|██████████| 54857/54857 [02:36<00:00, 350.57it/s]


In [25]:
# drop list with zero token
cnt_tkn = [len(l) for l in tokens_post]
cnt_tkn_serie = pd.Series(cnt_tkn)

index_drp = cnt_tkn_serie[cnt_tkn_serie==0].index.values

# drop original topic from top_df
df_post.drop(index_drp,inplace=True)

# drop from input list
for index in sorted(index_drp, reverse=True):
    del tokens_post[index]

In [26]:
print(len(tokens_post), len(df_post))
index_drp = df_post.loc[df_post.post_date.apply(len)!=19].index.values

# drop original topic from top_df
df_post.drop(index_drp,inplace=True)

# drop from input list
for index in sorted(index_drp, reverse=True):
    del tokens_post[index]
    
print(len(tokens_post), len(df_post))

54857 54857
54844 54844


In [27]:
df_post['post_date'] = pd.to_datetime(df_post['post_date'],errors='coerce')

print(len(tokens_post), len(df_post))
index_drp = df_post.loc[df_post.post_date.isna()].index.values

# drop original topic from top_df
df_post.drop(index_drp,inplace=True)

# drop from input list
for index in sorted(index_drp, reverse=True):
    del tokens_post[index]
    
print(len(tokens_post), len(df_post))

54844 54844
54844 54844


In [28]:
df_post.post_date.describe()

count                   54844
unique                  54832
top       2014-10-10 11:00:39
freq                        2
first     2013-01-01 19:55:37
last      2018-12-10 23:27:23
Name: post_date, dtype: object

In [29]:
with open('../out/aviation/tokens_post.txt', 'wb') as fp:
    pickle.dump(tokens_post, fp)
    
with open('../out/aviation/tokens_post.txt', 'rb') as fp:
    tokens_post = pickle.load(fp)

In [30]:
df_post.to_csv('../out/aviation/df_post.csv',index=False)

df_post = pd.read_csv('../out/aviation/df_post.csv')